In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Read Data Function
def read_data(filename):
    
    # Retrieve Data
    data = pd.read_csv(filename)
    
    # Drop Columns
    clean = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Date'])
    
    # Drop Last Row
    clean = clean[:-1].copy()
    
    # Drop missing data
    clean = clean.dropna()
    
    # Rename Columns
    cols = ['Year', 'Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']
    clean.columns = cols
    
    return clean

In [3]:
# Run Model (Machine Learning)
def run_model(clean):
    
    # Create Lag Column
    clean['nextHFC'] = clean.shift(1)['HFC']
    
    # Set Training and Testing Split
    training = clean.loc[(clean['Year']<2010) & (clean['Year']>1990)]
    testing = clean.loc[clean['Year']>=2010]

    # Set Variables
    X_train = training[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_train = training['nextHFC']
    X_test = testing[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_test = testing['nextHFC']
    X = clean[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y = clean['nextHFC']

    # Linear Regression Model
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()

    # Train Modele
    model.fit(X_train, y_train)

    # Score for 1991-2009
    print(f"Train Score: {model.score(X_train, y_train)}")

    # Score for 2010+
    print(f"Test Score: {model.score(X_test, y_test)}")

    # Predictions vs Actual
    predictions = model.predict(X)
    print(f"Actual: {y[1]}")
    print(f"Predicted: {predictions[1]}")
    print(f"Error: {predictions[1]-y[1]}")

    return pd.DataFrame({"Year": clean["Year"],
                         "Predicted": predictions,
                         "Actual": y,
                         "Error": predictions - y,
                         "Country": country})

In [4]:
final = pd.DataFrame(columns = ["Year", "Predicted", "Actual", "Error", "Country"])

for filename in os.listdir("Cleaning/Combined"):
    if ".csv" in filename:
        
        print()
        
        country = filename.replace('.csv', '')
        
        print(country)
        
        clean = read_data("Cleaning/Combined/" + filename)
        
        try:
            output = run_model(clean)
            final = final.append(output)
        
        except:
            print("Error")


Australia
Train Score: 0.9777708864723919
Test Score: 0.8783826666389848
Actual: 1424.6835199999996
Predicted: 1307.5336139999888
Error: -117.14990600001079

Canada
Train Score: 0.9957220550475632
Test Score: -70.52320516876284
Actual: 970.540192
Predicted: 1065.5854682453792
Error: 95.04527624537911

Japan
Train Score: 0.9987026092969515
Test Score: 0.9006539504039434
Actual: 444.0
Predicted: 421.8838321193216
Error: -22.116167880678404

New Zealand
Train Score: 0.9206598606986577
Test Score: -22.942839153960794
Actual: 5606.3288
Predicted: 4372.937989775717
Error: -1233.390810224283

Russia
Train Score: 0.9919146190787093
Test Score: 0.6919776376537219
Actual: 0.18119293223939
Predicted: -16.75313630731489
Error: -16.93432923955428

Spain
Train Score: 0.9968760935270009
Test Score: -0.005574165808244658
Actual: 0.02478119072
Predicted: -11.776555442117342
Error: -11.801336632837343

Switzerland
Train Score: 0.9974697924185433
Test Score: -5.148411312792866
Actual: 6.487443229575732


In [5]:
# Export 
final.to_csv('Resources/ML_HFC.csv')